In [1]:
# !pip3 install sacrebleu

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
from sacrebleu.metrics import BLEU, CHRF, TER

bleu = BLEU(force = True)

In [4]:
import malaya

model = malaya.translation.ms_en.transformer(model = 'small-bigbird')

/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.5.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(
/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/resource_loader.py:72: UserWarning: You are currently using TensorFlow 2.6.0 and trying to load a custom op (custom_ops/seq2seq/_beam_search_ops.so).
TensorFlow Addons has compiled its custom ops again

Downloading:   0%|          | 0.00/50.4M [00:00<?, ?B/s]

2022-07-21 18:07:10.884785: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-21 18:07:10.962882: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-21 18:07:10.975590: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-21 18:07:10.977904: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [5]:
with open('ms-en-right.test') as fopen:
    right = fopen.read().split('\n')
    
with open('ms-en-left.test') as fopen:
    left = fopen.read().split('\n')

In [6]:
from tqdm import tqdm

batch_size = 8

results = []
for i in tqdm(range(0, len(left), batch_size)):
    r_greedy = model.greedy_decoder(left[i:i + batch_size])
    results.extend(r_greedy)

 98%|████████████████████████████████████████████████████████████████████████████████████████▍ | 10757/10950 [30:25<00:43,  4.41it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
len(results)

87596

In [9]:
import re

def bigbird_postprocessing(string):
    return re.sub(r'([!,.?/;\'])\1+', r'\1 ', string).strip()

filtered_left, filtered_right = [], []
for no, r in enumerate(results):
    if len(r):
        filtered_left.append(bigbird_postprocessing(r))
        filtered_right.append(right[no])

In [10]:
refs = [filtered_right]
sys = filtered_left

In [11]:
r = bleu.corpus_score(sys, refs)

In [12]:
r.__dict__

{'name': 'BLEU',
 'score': 55.967144835516855,
 '_mean': -1.0,
 '_ci': -1.0,
 '_verbose': '77.4/60.5/49.9/41.9 (BP = 1.000 ratio = 1.026 hyp_len = 2053414 ref_len = 2001100)',
 'bp': 1.0,
 'counts': [1589954, 1189784, 937775, 751132],
 'totals': [2053414, 1965819, 1878357, 1791170],
 'sys_len': 2053414,
 'ref_len': 2001100,
 'precisions': [77.42978279100075,
  60.523578213457085,
  49.92528044455873,
  41.935271358944156],
 'prec_str': '77.4/60.5/49.9/41.9',
 'ratio': 1.026142621558143}